In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from utils.dataset import DataSet, LABELS
from models.own import OwnSingleModel
from keras.utils import to_categorical
from sklearn.metrics import f1_score, accuracy_score
from keras.utils import plot_model

from env import *

%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs/simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem.pkl')

MODEL_FILE = os.path.join(SAVED_MODELS_PATH, 'own/single.h5')
RESULT_CSV = os.path.join(RESULTS_PATH, 'own_single.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

seq = 'words_seq'
train_x = np.array(list(train_with_seq[seq]))
val_x = np.array(list(val_with_seq[seq]))
test_x = np.array(list(test_with_seq[seq]))

y_cols = LABELS
val_ys = []
train_outputs = []
val_outputs = []
for col in y_cols:
    train_y = train_with_seq[col] + 2
    val_y = val_with_seq[col] + 2
    val_ys.append(val_y)
    train_y_onehot = to_categorical(train_y)
    val_y_onehot = to_categorical(val_y)
    train_outputs.append(train_y_onehot)
    val_outputs.append(val_y_onehot)

In [6]:
model = OwnSingleModel(max_len = train_x.shape[1], embedding = embedding)
# model._model.summary()
plot_model(model._model)

In [ ]:
model.fit(train_x, train_outputs, 
          validation_data = (val_x, val_outputs),
          model_file = MODEL_FILE
         )

Train on 105000 samples, validate on 15000 samples
Epoch 1/300
105000/105000 [==============================] - 43s 411us/step - loss: 17.3045 - dense_1_loss: 0.6214 - dense_2_loss: 0.6562 - dense_3_loss: 0.7345 - dense_4_loss: 0.6362 - dense_5_loss: 1.1326 - dense_6_loss: 0.4425 - dense_7_loss: 0.7093 - dense_8_loss: 1.2208 - dense_9_loss: 0.8122 - dense_10_loss: 1.0081 - dense_11_loss: 0.9598 - dense_12_loss: 0.8120 - dense_13_loss: 0.9823 - dense_14_loss: 0.8724 - dense_15_loss: 1.1439 - dense_16_loss: 1.0360 - dense_17_loss: 0.9189 - dense_18_loss: 0.8093 - dense_19_loss: 0.9100 - dense_20_loss: 0.8862 - val_loss: 14.0445 - val_dense_1_loss: 0.3644 - val_dense_2_loss: 0.5289 - val_dense_3_loss: 0.4880 - val_dense_4_loss: 0.4968 - val_dense_5_loss: 0.9122 - val_dense_6_loss: 0.3046 - val_dense_7_loss: 0.5404 - val_dense_8_loss: 1.0961 - val_dense_9_loss: 0.6175 - val_dense_10_loss: 0.8034 - val_dense_11_loss: 0.7384 - val_dense_12_loss: 0.6410 - val_dense_13_loss: 0.8444 - val_dense

105000/105000 [==============================] - 41s 387us/step - loss: 9.1785 - dense_1_loss: 0.2155 - dense_2_loss: 0.3017 - dense_3_loss: 0.3062 - dense_4_loss: 0.3539 - dense_5_loss: 0.5019 - dense_6_loss: 0.1071 - dense_7_loss: 0.3260 - dense_8_loss: 0.6360 - dense_9_loss: 0.3489 - dense_10_loss: 0.4505 - dense_11_loss: 0.4984 - dense_12_loss: 0.5064 - dense_13_loss: 0.6733 - dense_14_loss: 0.4877 - dense_15_loss: 0.6986 - dense_16_loss: 0.6664 - dense_17_loss: 0.6028 - dense_18_loss: 0.3979 - dense_19_loss: 0.5921 - dense_20_loss: 0.5073 - val_loss: 9.4894 - val_dense_1_loss: 0.2192 - val_dense_2_loss: 0.3243 - val_dense_3_loss: 0.3216 - val_dense_4_loss: 0.3615 - val_dense_5_loss: 0.5219 - val_dense_6_loss: 0.1084 - val_dense_7_loss: 0.3325 - val_dense_8_loss: 0.6660 - val_dense_9_loss: 0.3584 - val_dense_10_loss: 0.4660 - val_dense_11_loss: 0.5166 - val_dense_12_loss: 0.5279 - val_dense_13_loss: 0.6700 - val_dense_14_loss: 0.5005 - val_dense_15_loss: 0.7346 - val_dense_16_loss:

105000/105000 [==============================] - 41s 387us/step - loss: 7.8105 - dense_1_loss: 0.1789 - dense_2_loss: 0.2575 - dense_3_loss: 0.2537 - dense_4_loss: 0.3180 - dense_5_loss: 0.4237 - dense_6_loss: 0.0933 - dense_7_loss: 0.2633 - dense_8_loss: 0.5326 - dense_9_loss: 0.3143 - dense_10_loss: 0.4016 - dense_11_loss: 0.4270 - dense_12_loss: 0.4178 - dense_13_loss: 0.5525 - dense_14_loss: 0.4126 - dense_15_loss: 0.5944 - dense_16_loss: 0.5629 - dense_17_loss: 0.5298 - dense_18_loss: 0.3337 - dense_19_loss: 0.5073 - dense_20_loss: 0.4356 - val_loss: 8.9594 - val_dense_1_loss: 0.2066 - val_dense_2_loss: 0.3209 - val_dense_3_loss: 0.3094 - val_dense_4_loss: 0.3401 - val_dense_5_loss: 0.4934 - val_dense_6_loss: 0.0987 - val_dense_7_loss: 0.2913 - val_dense_8_loss: 0.6260 - val_dense_9_loss: 0.3465 - val_dense_10_loss: 0.4546 - val_dense_11_loss: 0.4910 - val_dense_12_loss: 0.4742 - val_dense_13_loss: 0.6064 - val_dense_14_loss: 0.4713 - val_dense_15_loss: 0.7053 - val_dense_16_loss:

105000/105000 [==============================] - 41s 387us/step - loss: 6.7316 - dense_1_loss: 0.1542 - dense_2_loss: 0.2176 - dense_3_loss: 0.2137 - dense_4_loss: 0.2888 - dense_5_loss: 0.3616 - dense_6_loss: 0.0836 - dense_7_loss: 0.2323 - dense_8_loss: 0.4565 - dense_9_loss: 0.2853 - dense_10_loss: 0.3562 - dense_11_loss: 0.3668 - dense_12_loss: 0.3528 - dense_13_loss: 0.4646 - dense_14_loss: 0.3484 - dense_15_loss: 0.5132 - dense_16_loss: 0.4700 - dense_17_loss: 0.4627 - dense_18_loss: 0.2876 - dense_19_loss: 0.4360 - dense_20_loss: 0.3798 - val_loss: 8.9413 - val_dense_1_loss: 0.2067 - val_dense_2_loss: 0.3284 - val_dense_3_loss: 0.3120 - val_dense_4_loss: 0.3312 - val_dense_5_loss: 0.4912 - val_dense_6_loss: 0.0943 - val_dense_7_loss: 0.2825 - val_dense_8_loss: 0.6252 - val_dense_9_loss: 0.3469 - val_dense_10_loss: 0.4562 - val_dense_11_loss: 0.4924 - val_dense_12_loss: 0.4590 - val_dense_13_loss: 0.5903 - val_dense_14_loss: 0.4665 - val_dense_15_loss: 0.7127 - val_dense_16_loss:

In [ ]:
model._model.load_weights(MODEL_FILE)
val_probs = model.predict(val_x)
test_probs = model.predict(test_x)

val_preds = list(map(lambda x: np.argmax(x, axis = -1), val_probs))
# val_preds = np.argmax(val_probs, axis = 1)
# test_preds = list(map(lambda x: np.argmax(x, axis = -1), test_probs))

In [ ]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_ys, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))